<a href="https://colab.research.google.com/github/Servir-Mekong/data-driven-optical-sar-data-fusion/blob/main/error_metric_time_series.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import ee

try:
    ee.Initialize()
except:
    ee.Authenticate()
    ee.Initialize()


In [ ]:
%pylab inline

In [ ]:
import pandas as pd

In [ ]:
# function to provide a fill value if reduction is null
def filler(k,v):
    newval = ee.Algorithms.If(v,v,-9999)
    return newval

In [ ]:
# function to calculate error metrics for each date in collection
def calc_metrics(img):
    date = img.get("system:time_start")
    img = img.divide(1e5)
    y_true = img.select([0])
    y_pred = img.select([1])
    bias = y_pred.subtract(y_true).rename("error")
    se = bias.pow(2).rename("sqr_error")
    pe = y_true.subtract(y_pred).divide(y_true).abs().rename("pct_error")
    metrics = ee.Image.cat([
        bias,
        se,
        pe
    ]).set("system:time_start",date)

    metric_dict = metrics.reduceRegion(
        reducer= ee.Reducer.mean(),
        geometry= img.geometry(),
        scale= 30,
        maxPixels= 1e10
    ).map(filler)

    return metrics.set(metric_dict)

In [ ]:
# collection that is 
collection = ee.ImageCollection("users/kelmarkert/optical_sar_df_coincident").sort("system:time_start")

In [ ]:
metrics = collection.map(calc_metrics)

In [ ]:
dates = np.array(
    metrics.aggregate_array("system:time_start").getInfo()
)

In [ ]:
bias = np.array(
    metrics.aggregate_array("error").getInfo()
)

bias[bias==-9999.0] = np.nan

In [ ]:
sqr_err = np.array(
    metrics.aggregate_array("sqr_error").getInfo()
)

In [ ]:
pct_err = np.array(
    metrics.aggregate_array("pct_error").getInfo()
)

In [ ]:
rmse = np.sqrt(sqr_err)

In [ ]:
df = pd.DataFrame(
    {
        "bias": bias,
        "mse": sqr_err,
        "rmse": rmse,
        "mape": pct_err
    },
    index=pd.to_datetime(dates*1e6)
)

df.dropna(inplace=True)

In [ ]:
df

In [ ]:

df.plot(marker="o",subplots=True, figsize=(10,10));